## Magics

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, roc_auc_score, f1_score, classification_report

In [ ]:
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset, BankDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
                import load_preproc_data_adult, load_preproc_data_compas

from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression

from IPython.display import Markdown, display

In [ ]:
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## Get Data

In [ ]:
## import dataset
dataset_used = "adult" # "adult", "german", "compas"
protected_attribute_used = 1 # 1, 2

if dataset_used == "adult":
    dataset_orig = AdultDataset()
#     dataset_orig = load_preproc_data_adult()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
    
elif dataset_used == "german":
    dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
    
elif dataset_used == "compas":
#     dataset_orig = CompasDataset()
    dataset_orig = load_preproc_data_compas()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]    

In [ ]:
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.6], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

In [ ]:
# print out some labels, names, etc.
display(Markdown("#### Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale_orig = StandardScaler()
X_train = torch.tensor(scale_orig.fit_transform(dataset_orig_train.features), dtype=torch.float32)
y_train = torch.tensor(dataset_orig_train.labels.ravel(), dtype=torch.float32)


X_valid = torch.tensor(scale_orig.transform(dataset_orig_valid.features), dtype=torch.float32)
y_valid = torch.tensor(dataset_orig_valid.labels.ravel(), dtype=torch.float32)


X_test = torch.tensor(scale_orig.transform(dataset_orig_test.features), dtype=torch.float32)
y_test = torch.tensor(dataset_orig_test.labels.ravel(), dtype=torch.float32)

## Deep Learning Model

In [ ]:
class Model(nn.Module):
    
    def __init__(self, input_size, num_deep=10, hid=32, dropout_p=0.2):
        super().__init__()
        self.fc0 = nn.Linear(input_size, hid)
        self.bn0 = nn.BatchNorm1d(hid)
        self.fcs = nn.ModuleList([nn.Linear(hid, hid) for _ in range(num_deep)])
        self.bns = nn.ModuleList([nn.BatchNorm1d(hid) for _ in range(num_deep)])
        self.out = nn.Linear(hid, 2)
        self.dropout = nn.Dropout(dropout_p)
        
    def forward(self, t):
        t = self.bn0(self.dropout(F.relu(self.fc0(t))))
        for bn, fc in zip(self.bns, self.fcs):
            t = bn(self.dropout(F.relu(fc(t))))
        return torch.sigmoid(self.out(t))
    
    def trunc_forward(self, t):
        t = self.bn0(self.dropout(F.relu(self.fc0(t))))
        for bn, fc in zip(self.bns, self.fcs):
            t = bn(self.dropout(F.relu(fc(t))))
        return t
    
model = Model(dataset_orig_train.features.shape[1])
loss_fn = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
patience = (math.inf, None, 0)
patience_limit = 4
for epoch in range(201):
    model.train()
    batch_idxs = torch.split(torch.randperm(X_train.size(0)), 64)
    train_loss = 0
    for batch in batch_idxs:
        X = X_train[batch,:]
        y = y_train[batch]
        optimizer.zero_grad()
        loss = loss_fn(model(X)[:,0], y)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            valid_loss = loss_fn(model(X_valid)[:,0], y_valid)
        if valid_loss > patience[0]:
            patience = (patience[0], patience[1], patience[2]+1)
        else:
            patience = (valid_loss, model.state_dict(), 0)
        if patience[2] > patience_limit:
            print("Ending early, patience limit has been crossed without an increase in validation loss!")
            model.load_state_dict(patience[1])
            break
        print(f'=======> Epoch: {epoch} Train loss: {train_loss / len(batch_idxs)} Valid loss: {valid_loss} Patience valid loss: {patience[0]}')

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, classification_report

model.eval()
with torch.no_grad():
    y_valid_hat = model(X_valid)[:,0]

fpr, tpr, thresh = roc_curve(y_valid, y_valid_hat)
roc_auc = roc_auc_score(y_valid, y_valid_hat)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

x = np.linspace(0,1,1000)
fscore = [f1_score(y_valid, y_valid_hat > i) for i in x]
plt.plot(x, fscore)
plt.show()

best_thresh = x[np.argmax(fscore)]
print(f'Threshold to maximize f1 score is {best_thresh}')
print(classification_report(y_valid, y_valid_hat > best_thresh))

## Linear Model (Logistic Regression)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

# Logistic regression classifier and predictions for training data]
lmod = LogisticRegression()
lmod.fit(X_train, y_train)

from sklearn.metrics import roc_curve, roc_auc_score, f1_score
# Prediction probs for validation and testing data
X_valid = torch.tensor(scale_orig.transform(dataset_orig_valid.features), dtype=torch.float32)
y_valid_hat = lmod.predict_proba(X_valid)[:, 1]
y_valid = dataset_orig_valid.labels.ravel()

fpr, tpr, thresh = roc_curve(y_valid, y_valid_hat)
roc_auc = roc_auc_score(y_valid, y_valid_hat)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

x = np.linspace(0,1,1000)
fscore = [f1_score(y_valid, y_valid_hat > i) for i in x]
plt.plot(x, fscore)
plt.show()


best_thresh = x[np.argmax(fscore)]
print(f'Threshold to maximize f1 score is {best_thresh}')
print(classification_report(y_valid, y_valid_hat > best_thresh))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_valid_hat = rf.predict_proba(X_valid)[:,1]
fpr, tpr, thresh = roc_curve(y_valid, y_valid_hat)
roc_auc = roc_auc_score(y_valid, y_valid_hat)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

x = np.linspace(0,1,1000)
fscore = [f1_score(y_valid, y_valid_hat > i) for i in x]
plt.plot(x, fscore)
plt.show()


best_thresh = x[np.argmax(fscore)]
print(f'Threshold to maximize f1 score is {best_thresh}')
print(classification_report(y_valid, y_valid_hat > best_thresh))

## Initial Bias

In [ ]:
from collections import OrderedDict
from aif360.metrics import ClassificationMetric

def compute_metrics(dataset_true, dataset_pred, 
                    unprivileged_groups, privileged_groups,
                    disp = True):
    """ Compute the key metrics """
    classified_metric_pred = ClassificationMetric(dataset_true,
                                                 dataset_pred, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
    metrics["Theil index"] = classified_metric_pred.theil_index()
    
    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))
    
    return metrics

In [ ]:
model.eval()
with torch.no_grad():
    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.scores = model(X_train)[:,0].reshape(-1,1).numpy()

    dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    dataset_orig_valid_pred.scores = model(X_valid)[:,0].reshape(-1,1).numpy()

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    dataset_orig_test_pred.scores = model(X_test)[:,0].reshape(-1,1).numpy()

In [ ]:
# Transform the validation set
model.eval()
with torch.no_grad():
    dataset_transf_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    dataset_transf_valid_pred.labels = (model(X_valid)[:,0] > best_thresh).reshape(-1,1).numpy()

display(Markdown("#### Validation set - Initial"))
display(Markdown("##### Transformed predictions"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred, unprivileged_groups, privileged_groups)

display(Markdown("##### Classification Report"))
print(classification_report(y_valid, dataset_transf_valid_pred.labels))

## Reject Option Classification

In [ ]:
allowed_metrics = ["Statistical parity difference", "Average odds difference", "Equal opportunity difference"]

In [ ]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name="Statistical parity difference",
                                  metric_ub=0.05, metric_lb=-0.05)
ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

roc_thresh = ROC.classification_threshold

print("Optimal classification threshold (with fairness constraints) = %.4f" % roc_thresh)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

In [ ]:
# Transform the validation set
dataset_transf_valid_pred = ROC.predict(dataset_orig_valid_pred)

display(Markdown("#### Validation set - With ROC fairness"))
display(Markdown("##### Transformed predictions"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred, unprivileged_groups, privileged_groups)

display(Markdown("##### Classification Report"))
print(classification_report(y_valid, dataset_transf_valid_pred.labels))

## Calibrated Equalized Odds

In [ ]:
# Odds equalizing post-processing algorithm
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from tqdm import tqdm


# cost constraint of fnr will optimize generalized false negative rates, that of
# fpr will optimize generalized false positive rates, and weighted will optimize
# a weighted combination of both
cost_constraint = "fnr" # "fnr", "fpr", "weighted"
randseed=101


# Learn parameters to equalize odds and apply to create a new dataset
cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint=cost_constraint,
                                     seed=randseed)
cpp = cpp.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [ ]:
dataset_transf_valid_pred = cpp.predict(dataset_orig_valid_pred)

display(Markdown("#### Validation sets - With CalibEqOdds fairness"))
display(Markdown("##### Transformed prediction"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred, unprivileged_groups, privileged_groups)

display(Markdown("##### Classification Report"))
print(classification_report(y_valid, dataset_transf_valid_pred.labels))

## Random

In [ ]:
# from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
# ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
#                                  privileged_groups=privileged_groups, 
#                                  low_class_thresh=0.01, high_class_thresh=0.99,
#                                   num_class_thresh=100, num_ROC_margin=50,
#                                   metric_name="Statistical parity difference",
#                                   metric_ub=0.05, metric_lb=-0.05)

results = []
for _ in range(50):
    rand_model = Model(X_train.size(1))
    rand_model.load_state_dict(model.state_dict())
    for param in rand_model.parameters():
        param.data = param.data * (torch.randn_like(param) + 1)

    rand_model.eval()
    with torch.no_grad():
        dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
        dataset_orig_valid_pred.scores = rand_model(X_valid)[:,0].reshape(-1,1).numpy()
    
    roc_auc = roc_auc_score(dataset_orig_valid.labels, dataset_orig_valid_pred.scores)
    threshs = np.linspace(0,1,101)
    fscores = []
    for thresh in threshs:
        fscores.append(f1_score(dataset_orig_valid.labels, dataset_orig_valid_pred.scores > thresh))
    best_rand_thresh = threshs[np.argmax(fscores)]
    dataset_orig_valid_pred.labels = dataset_orig_valid_pred.scores > best_rand_thresh

#     ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

#     dataset_transf_valid_pred = ROC.predict(dataset_orig_valid_pred)
    classified_metric_pred = ClassificationMetric(dataset_orig_valid, dataset_orig_valid_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    spd = classified_metric_pred.statistical_parity_difference()
    results.append([roc_auc, spd])

In [ ]:
sorted(results)

## Critic


In [ ]:
class Critic(nn.Module):
    
    def __init__(self, sizein, num_deep=3, hid=32):
        super().__init__()
        self.fc0 = nn.Linear(sizein, hid)
        self.fcs = nn.ModuleList([nn.Linear(hid, hid) for _ in range(num_deep)])
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(hid, 1)
        
    def forward(self, t):
        t = t.reshape(1,-1)
        t = self.fc0(t)
        for fc in self.fcs:
            t = F.relu(fc(t))
            t = self.dropout(t)
        return self.out(t)

In [ ]:
truncmodel = Model(X_train.size(1))
truncmodel.load_state_dict(model.state_dict())
critic = Critic(1024)

In [ ]:
trunc_optimizer = optim.Adam(truncmodel.parameters())
critic_optimizer = optim.Adam(critic.parameters())
critic_loss_fn = torch.nn.MSELoss()

In [ ]:
for epoch in range(51):
    
    for param in critic.parameters():
        param.requires_grad = True
    for param in truncmodel.parameters():
        param.requires_grad = False
    truncmodel.eval()
    critic.train()
    for step in range(201):
        critic_optimizer.zero_grad()
        indices = torch.randint(0, X_valid.size(0), (32,))
        cy_valid = y_valid[indices]
        cX_valid = X_valid[indices]
        with torch.no_grad():
            dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
            dataset_orig_valid_pred.scores = truncmodel(cX_valid)[:,0].reshape(-1,1).numpy()
        
        classified_metric_pred = ClassificationMetric(dataset_orig_valid, dataset_orig_valid_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        spd = classified_metric_pred.statistical_parity_difference()
        
        res = critic(truncmodel.trunc_forward(cX_valid))
        loss = critic_loss_fn(torch.tensor(spd), res[0])
        loss.backward()
        train_loss = loss.item()
        critic_optimizer.step()
        if step % 100 == 0:
            print(f'=======> Epoch: {(epoch, step)} Critic loss: {train_loss}')
            
    for param in critic.parameters():
        param.requires_grad = False
    for param in truncmodel.parameters():
        param.requires_grad = True
    truncmodel.train()
    critic.eval()
    for step in range(101):
        trunc_optimizer.zero_grad()
        indices = torch.randint(0, X_valid.size(0), (32,))
        cy_valid = y_valid[indices]
        cX_valid = X_valid[indices]
        
        bias = abs(critic(truncmodel.trunc_forward(cX_valid)))
        loss = critic_loss_fn(cy_valid, truncmodel(cX_valid)[:,0])
        loss = loss + 2*bias
        
        loss.backward()
        train_loss = loss.item()
        trunc_optimizer.step()
        if step % 100 == 0:
            print(f'=======> Epoch: {(epoch, step)} Actor loss: {train_loss}')

In [ ]:
with torch.no_grad():
    dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    dataset_orig_valid_pred.scores = truncmodel(X_valid)[:,0].reshape(-1,1).numpy()

roc_auc = roc_auc_score(dataset_orig_valid.labels, dataset_orig_valid_pred.scores)
threshs = np.linspace(0,1,1001)
fscores = []
for thresh in threshs:
    fscores.append(f1_score(dataset_orig_valid.labels, dataset_orig_valid_pred.scores > thresh))
best_rand_thresh = threshs[np.argmax(fscores)]
dataset_orig_valid_pred.labels = dataset_orig_valid_pred.scores > best_rand_thresh

display(Markdown("#### Validation sets - With Critic fairness"))
display(Markdown("##### Transformed prediction"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_orig_valid_pred, unprivileged_groups, privileged_groups)

display(Markdown("##### Classification Report"))
print(classification_report(y_valid, dataset_transf_valid_pred.labels))

In [ ]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name="Statistical parity difference",
                                  metric_ub=0.05, metric_lb=-0.05)
with torch.no_grad():
    dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    dataset_orig_valid_pred.scores = truncmodel(X_valid)[:,0].reshape(-1,1).numpy()
    

ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)
roc_thresh = ROC.classification_threshold

# Transform the validation set
dataset_transf_valid_pred = ROC.predict(dataset_orig_valid_pred)

display(Markdown("#### Validation set - With Critic + ROC fairness"))
display(Markdown("##### Transformed predictions"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred, unprivileged_groups, privileged_groups)

display(Markdown("##### Classification Report"))
print(classification_report(y_valid, dataset_transf_valid_pred.labels))